In [9]:
import rasterio
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
path = "MODIS_AOD_Milan_2025_Jan_to_NovMultiBand.tif"

with rasterio.open(path) as src:
    data = src.read() 
    nbands, nrows, ncols = data.shape
    rows, cols = np.meshgrid(np.arange(nrows), np.arange(ncols), indexing="ij")
    xs, ys = rasterio.transform.xy(src.transform, rows, cols)

In [3]:
flat = data.reshape(nbands, -1).T

In [4]:
band_cols = [f"band_{i+1}" for i in range(nbands)]
df = pd.DataFrame(flat, columns=band_cols)

In [5]:
df["x"] = np.array(xs).ravel()
df["y"] = np.array(ys).ravel()

In [6]:
df

,band_1,band_2,band_3,band_4,band_5,band_6,band_7,band_8,band_9,band_10,...,band_320,band_321,band_322,band_323,band_324,band_325,band_326,band_327,x,y
0,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,114.0,...,NaN,NaN,40.0,NaN,NaN,NaN,43.5,222.0,9.104425,45.522127
1,NaN,NaN,24.5,NaN,NaN,NaN,NaN,NaN,NaN,110.0,...,NaN,NaN,39.0,NaN,NaN,NaN,41.5,228.0,9.113409,45.522127
2,NaN,NaN,24.5,NaN,NaN,NaN,NaN,NaN,NaN,110.0,...,NaN,NaN,39.0,NaN,NaN,NaN,41.5,228.0,9.122392,45.522127
3,NaN,NaN,33.0,NaN,NaN,NaN,NaN,NaN,NaN,107.0,...,NaN,NaN,31.0,NaN,NaN,NaN,41.5,NaN,9.131375,45.522127
4,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,98.0,...,NaN,NaN,31.0,NaN,NaN,NaN,43.5,NaN,9.140358,45.522127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,NaN,NaN,42.5,NaN,NaN,NaN,NaN,NaN,NaN,130.0,...,NaN,96.0,14.0,NaN,NaN,NaN,47.0,202.0,9.266122,45.351447
456,NaN,NaN,47.5,NaN,NaN,NaN,NaN,NaN,NaN,126.0,...,NaN,94.0,18.0,NaN,NaN,NaN,46.5,207.0,9.275105,45.351447
457,NaN,NaN,47.5,NaN,NaN,NaN,NaN,NaN,NaN,126.0,...,NaN,94.0,18.0,NaN,NaN,NaN,46.5,207.0,9.284088,45.351447
458,NaN,NaN,47.5,NaN,NaN,NaN,NaN,NaN,NaN,119.0,...,NaN,93.0,27.0,NaN,NaN,NaN,46.0,206.0,9.293072,45.351447


In [7]:
target_x, target_y = 9.20, 45.51

df['distance'] = np.sqrt((df['x'] - target_x)**2 + (df['y'] - target_y)**2)

closest_pixel = df.loc[df['distance'].idxmin()]

In [8]:
closest_pixel

band_1             NaN
band_2             NaN
band_3       26.000000
band_4             NaN
band_5             NaN
               ...    
band_326     41.000000
band_327    149.000000
x             9.203240
y            45.513144
distance      0.004515
Name: 34, Length: 330, dtype: float64

In [17]:
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 11, 23)
dates = pd.date_range(start=start_date, end=end_date, freq='D')[:len(band_cols)]

# Get band columns and AOD values
band_cols = [col for col in closest_pixel.index if col.startswith('band_')]
aod_values = [closest_pixel[col] for col in band_cols]

# Simple 2-column DataFrame
aod_df = pd.DataFrame({
    'date': dates,
    'AOD': aod_values
})

print(aod_df.head())

        date   AOD
0 2025-01-01   NaN
1 2025-01-02   NaN
2 2025-01-03  26.0
3 2025-01-04   NaN
4 2025-01-05   NaN


In [18]:
aod_df.to_csv('modis_aod_gigliacci_milano.csv', index=False)